Created on Mon Oct 11 14:54:29 2021

@author: Gregoire, Adrien

# Imports

In [1]:
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from matplotlib.collections import PolyCollection
from matplotlib import colors as mcolors
from matplotlib.patches import FancyArrowPatch

def polygon_under_graph(xlist, ylist):
    '''
    Construct the vertex list which defines the polygon filling the space under
    the (xlist, ylist) line graph.  Assumes the xs are in ascending order.
    '''
    return [(xlist[0], 0.), *zip(xlist, ylist), (xlist[-1], 0.)]
def cc(arg,alpha):
    '''
    Shorthand to convert 'named' colors to rgba format at 60% opacity.
    '''
    return mcolors.to_rgba(arg, alpha)

In [3]:
%matplotlib qt
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

In [4]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

hbar = 1.0545718e-34
c = 299792458

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = (4e-3)/2            # longueur piège Ca+
Omega = 2.045e6*2*pi # 2.047e6
bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Computation of Probabilities
$\texttt{MB}$ is the Maxwell-Boltzmann distribution. It provides the probability to find an atom with a given velocity in a gas with temperature T.

$\texttt{pfl_dop}$ is the atomic ray profile with Doppler effect. It provides the probability of excitation given Rabi frequency, detuning, lambda and velocity of atom. It is considered equal to the excited population, i.e the proportion of excited atoms.

The fluorescence F is related to the product of both probabilities $ F = \propto \texttt{MB}\times \texttt{pfl_dop} = \texttt{prob_fluo}$. This product is still a probability, lets call it $P$. The idea is to search for the setting where the variation of fluorescence is the highest for a given input in energy. The energy introduced by the GiantMolecule is brought under the form of thermal energy. Thus we want to set the cloud in a configuration with the best variation of fluorescence for a variation in temperature. There are two ways of achieving this. First considering infinitesimal differences, this requires to compute $dP/dT$. Second considering macroscopic differences $\Delta P$ and $\Delta T$. Because we know how much energy the GiantMolecule transfers to the cloud, we can compute the expected $\Delta T$. Then search for the optimal $\Delta P$ given the determines $\Delta T$. In the simulation article, the GMol with incident energy 50eV, transfers 50meV, which produces a 100mK increase of temperature in the cloud.

In [11]:
# Doppler profile
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38e-23
    m_Ca = 40*1.66e-27
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [178]:
# Laser parameters
k = 15826663 # m^-1
Gam = 21570000.0 *2*pi
delta = -Gam
lam = 397e-9 # m
I = 170 # W/m²

# First method with infinitesimal differences

Consider the excited state probability $P = \rho_{ee} = \frac{A/2}{A + B + (\delta - v)^2}$, with $A=\frac{1}{2}\Omega_{rabi}^2=\frac{1}{2}\frac{3\lambda\Gamma}{4\pi^2\hbar c}I_L$, $B=\frac{\Gamma^2}{4}$. The derivation provides us with the following expressions
- $\frac{dP}{dT} = \frac{A}{2}\frac{2(\delta - v)}{(A+B+(\delta - v)^2)^2}$
- $\frac{d^2P}{dT^2} \propto \frac{A}{2}(-2(A+B+z)^2+8z(A+B+z))$, with $z=(\delta - v)^2$.

$\frac{d^2P}{dT^2} = 0 \Leftrightarrow 3z^2+4(A+B)z-2(A+B)^2 = 0$.

This provides us with an expression for $z = \delta - v = \pm \sqrt{\frac{A+B}{3}}$. So $v = \delta - z$.
Ultimately, considering the most probable speed in a gas is $v = \sqrt{2k_BT/m}$ it is possible to rewrite $v = \delta - z$ as

$T = \frac{m(\delta - z)^2}{2k_B}$

In [7]:
T = 0.1
v = sqrt(2*kb*T/m_Ca)
s = 2 # dans le code 2 correspond a omega_rabi=Gam
s0 = 1
omega_rabi = sqrt(2*s*(0.25*Gam**2+(delta-k*v)**2))
# omega_rabi = sqrt( 2*s0*0.25*Gam**2 )
A = 0.5*omega_rabi**2
B = 0.25*Gam**2
zz = sqrt( (A+B)/3 )
T = m_Ca*(delta-zz)**2/(2*kb)/k**2
print(T)

1.1183600415127386


In [8]:
np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I),Gam

(58865055.550918736, 135528307.0758637)

In [9]:
Gam,delta

(135528307.0758637, -135528307.0758637)

In [182]:
# Plot MB dist and Dop profile as probabilities

T = .001
nu = k*np.linspace(-20,20,10000)
dv = mean(diff(nu))/k
y = MB(nu/k, T)

Rab = np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) # Gam 

pfl = pfl_dop(nu/k, delta, k, Rab, Gam)
pfl2 = pfl_dop(nu/k, -delta, k, Rab, Gam)

fig = plt.figure('MB and Dop prof',clear='True')
ax = fig.add_subplot(111)
ax.plot(nu*1e-6, y/np.max(y),label=f'{T*1e3:.1f} mK MB profile (normalised)')
ax.plot(nu*1e-6, pfl/np.max(pfl), '-',label='Doppler profile (normalised)')
ax.plot(nu*1e-6, pfl2/np.max(pfl2), '-',label='Doppler profile (normalised)')

ax.arrow(-Gam*1e-6+10, 1.05,
         Gam*1e-6-20, 0,
         color='xkcd:rouge',
         width=1e-2,head_width=5e-2,head_length=10,zorder=100)
ax.arrow(-10, 1.05,
         -Gam*1e-6+20, 0,
         color='xkcd:rouge',
         width=1e-2,head_width=5e-2,head_length=10)

ax.set_xlabel(r'$\nu = k v$ [MHz]')
ax.set_ylabel(r'Probability')
ax.set_ylim(-0.05,1.25)
ax.annotate(r'$\Gamma$', xy=(-Gam*1e-6/2,1.075), xycoords='data',
    size=25, ha='center', va='bottom', color='xkcd:rouge')

below = []
for i in range(len(y)):
    if y[i]/np.max(y)>pfl[i]/np.max(pfl):
        below.append(pfl[i]/np.max(pfl))
    else:
        below.append(y[i]/np.max(y))
verts1 = []
verts1.append(polygon_under_graph(nu*1e-6,below))

poly1 = PolyCollection(verts1, facecolors=cc('xkcd:purplish blue',0.85), hatch = '..')
ax.add_collection(poly1)

ax.grid()
ax.legend(loc=1,fontsize=15)

tight_layout()

In [129]:
dv

0.04004004004004005

In [13]:
# plot P = MB x DopProfile

vmin = -20 # from 
vmax = 20  # to
nv = 1000  # resolution
T = 0.001    # temperature

nunu, dop = prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam)

dv = mean(diff(nunu))/k

fig = plt.figure('MB x Dop',clear='True')
ax = fig.add_subplot(111)
ax.plot(nunu*1e-6, dop)

ax.set_xlabel(r'$\nu = k v$ [MHz]')
ax.set_ylabel(r'Probability')

ax.grid()
ax.legend(loc=3,fontsize=15)

tight_layout()

No handles with labels found to put in legend.


In [149]:
# For a range of temperatures
# Integrate the product MBxDopProfile
# For several laser detunings
# >>> Takes one minute <<<

temps = np.linspace(.00055, 5, 10000)  # temperatures
deltas = np.linspace(0, 2, 51)*Gam # detunings
# deltas = hstack((np.arange(0, 0.33, 0.02)*Gam,
#                 np.linspace(0.33, 0.35, 20)*Gam,
#                 np.arange(0.35, 2, 0.02)*Gam))  # 0.31 0.35
z = [] # Here is stored the integral MBxDopProfile over detunings
for i, temp in enumerate(temps):
    z.append([])    
    for j, delta in enumerate(deltas):
        nunu, dop = prob_fluo(vmin, vmax, nv, temp, delta, k, Rab, Gam)
        z[i].append(np.sum(dop)*dv) # the integrals are normalised
                                    # by dv the step in v

In [195]:
# Plot z
# the excited population

fig = plt.figure('ex curve 0',clear='True')
ax = fig.add_subplot(111)
# plt.title('Optimal temperature')
for_detuning = 50
print(deltas[for_detuning]/Gam)

ax.plot(temps*1e3, array(z)[:,for_detuning],
        marker='',ls='-',color = my_cm(0/3))

ax.set_xlabel('Temperature [mK]')
ax.set_ylabel('P')
ax.grid()
# ax.legend(fontsize=15)
tight_layout()

2.0


In [15]:
# Plot those integrals

my_cm = plt.get_cmap('jet')

n = np.shape(z)[1]

fig = plt.figure('Integral',clear='True')
ax = fig.add_subplot(111)
for i in np.arange(0,n,2): # only show one curve over two
    ax.plot(temps*1000, np.array(z)[:,i], color = my_cm(i/n))
ax.text(25,0.15,'One color for each detuning',fontsize=15)
ax.set_xlabel('Temperature [mK]')
ax.set_ylabel('Probabilities')
ax.grid()
tight_layout()

In [16]:
# Search integrals maximum
# with derivative method

n = np.shape(z)[1]
fig = plt.figure('Integral derivate',clear='True')
ax = fig.add_subplot(111)
for i in np.arange(0,n//2,2):
    ax.plot(temps[:-1], np.abs(np.diff(np.array(z)[:,i])), color = my_cm(i/n) )
ax.set_xlabel('Temperature [mK]')
ax.grid()
tight_layout()

In [17]:
cut_bif = 36

In [167]:
# Search integrals maximum
# with derivative method
# pos of optimal temp

# z[temperature,detuning] is the integral MBxDopProfile

n = np.shape(z)[1]
T0 = []
F0 = []
for i in np.arange(0,n,1):
    T0.append([])
    F0.append([])
    T0[i].append(temps[np.argmin(np.abs(np.diff(np.array(z)[:,i])))])
    T0[i].append(temps[np.argmin(np.diff(np.array(z)[:,i]))])
    T0[i].append(temps[np.argmax(np.diff(np.array(z)[:,i]))])
    T0[i].append(temps[np.argmin(np.abs(np.diff(np.array(z)[:,i],n=2)))])
    F0[i].append(max(np.abs(np.diff(np.array(z)[:,i]))))
    F0[i].append(min(np.diff(np.array(z)[:,i])))
    F0[i].append(max(np.diff(np.array(z)[:,i])))
    
T0 = array(T0)
F0 = array(F0)

Temp_for_fluomax = T0[:,0]
Temp_for_fluovarmax = T0[:,3]

True_min = T0[:,1]
True_max = T0[:,2]
True_deltas = hstack((deltas[:],deltas[:]))
True_true = hstack((True_min,True_max))

Frue_min = F0[:,1]
Frue_max = F0[:,2]
Frue_true = hstack((Frue_min,Frue_max))

# ou Fluo est max c'est le min de la valeur absolue de diff(z)


In [191]:
# Plot one curve as example
# diff(np.array(z)[:,i]) vs temperature

fig = plt.figure('ex curve',clear='True')
ax = fig.add_subplot(111)
# plt.title('Optimal temperature')
for_detuning = 19
print(deltas[for_detuning]/Gam)

ax.plot(temps[:-1]*1e3, diff(np.array(z)[:,for_detuning]),
        marker='',ls='-',color = my_cm(0/3))

ax.set_xlabel('Temperature [mK]')
ax.set_ylabel('dP/dT')
ax.grid()
# ax.legend(fontsize=15)
tight_layout()

0.76


In [176]:
len(Temp_for_fluovarmax)

51

In [189]:
# Plot bifurcation diagram

# Search integrals maximum
# with derivative method
# pos of optimal temp diag bif
# Depending the detuning

fig = plt.figure('Bifurcation Temperature',clear='True')
ax = fig.add_subplot(111)
# plt.title('Optimal temperature')

# ax.plot(deltas/Gam, Temp_for_fluovarmax*1000,
#         marker='+',ls='--',color = my_cm(2/3)) # ,label='Opt. Temp.' 
ax.plot(deltas/Gam, Temp_for_fluomax*1000,
        marker='+',ls=':',color = my_cm(1/3)) # ,label='Opt. Temp.' 
# ax.plot(True_deltas[:100]/Gam, True_true[:100]*1000,
#         marker='.',ls='-',color = my_cm(0/3)) # ,label='Opt. Temp.' 
# ax.plot(True_deltas[117:]/Gam, True_true[117:]*1000,
#         marker='.',ls='-',color = my_cm(0/3)) # ,label='Opt. Temp.'

# ax.plot(True_deltas[100:]/Gam, True_true[100:]*1000,
#         marker='.',ls='-',color = my_cm(1/3)) # ,label='Opt. Temp.' 
# ax.plot(True_deltas[:cut_bif]/Gam, True_true[:cut_bif]*1000,
#         marker='',ls='-',color = my_cm(0/3))
# ax.plot(True_deltas[cut_bif:]/Gam, True_true[cut_bif:]*1000,
#         marker='',ls='-',color = my_cm(0/3))
ax.vlines(1,-15,950,linestyle=':')
# ax.plot(deltas/Gam,T_lim(-deltas)*1e3,'+',label='Dop. Lim. temperature')
    
ax.set_xlabel('Detuning [$\Gamma$]')
ax.set_ylabel('Temperature [mK]')
ax.grid()
# ax.legend(fontsize=15)
tight_layout()

In [53]:
# Plot bifurcation diagram

# Search integrals maximum
# with derivative method
# pos of optimal temp diag bif
# Depending the detuning

fig = plt.figure('Bifurcation Fluorescence',clear='True')
ax = fig.add_subplot(111)
# plt.title('Optimal temperature')

ax.plot(True_deltas[:100]/Gam, Frue_true[:100]*1000,
        marker='',ls='-',color = my_cm(0/3)) # ,label='Opt. Temp.' 
ax.plot(True_deltas[117:]/Gam, Frue_true[117:]*1000,
        marker='',ls='-',color = my_cm(0/3)) # ,label='Opt. Temp.'
# ax.plot(True_deltas[:cut_bif]/Gam, True_true[:cut_bif]*1000,
#         marker='',ls='-',color = my_cm(0/3))
# ax.plot(True_deltas[cut_bif:]/Gam, True_true[cut_bif:]*1000,
#         marker='',ls='-',color = my_cm(0/3))
# ax.vlines(1,-15,950,linestyle=':')
# ax.plot(deltas/Gam,T_lim(-deltas)*1e3,'+',label='Dop. Lim. temperature')
    
ax.set_xlabel('Detuning [$\Gamma$]')
ax.set_ylabel('Fluorescence [F/K]')
ax.grid()
# ax.legend(fontsize=15)
tight_layout()

In [20]:
fig = plt.figure('Integral derivate + extrema',clear='True')
ax = fig.add_subplot(111)
for i in np.arange(0,n,1):
    ax.plot(temps[:-1]*1e3, (np.diff(np.array(z)[:,i]))*1e3, color = my_cm(i/n) )
# ax.plot(T0[:,0]*1e3, F0[:,0],
#         marker='o',color = 'k' )
ax.plot(True_true*1e3, Frue_true*1e3,
        marker='+',ls='',ms=8,mew=3,color = 'k',label='Opt. Temp.' )
    
ax.set_xlabel('Temperature [mK]')
ax.set_ylabel(r'$dP/dT$ [$ \times $1000]')
ax.grid()
tight_layout()

In [21]:
# Search the temperature
# for which dP/dT is max

T0 = []
for i in np.arange(n):
    T0.append(temps[np.argmax(np.abs(np.diff(np.array(z)[:,i])))])

fig = plt.figure('Ideal temperature',clear='True')
ax = fig.add_subplot(111)
plt.plot(deltas/Gam*2, multiply(T0,1000), 'o')
ax.set_xlabel('Detuning [$\Gamma/2$]')
ax.set_ylabel('Temperature [mK]')
ax.grid()
tight_layout()

In [285]:
# computation ideal temperature
# as Jofre intended

A = .5*Rab**2
B = .25*Gam**2
deltamoinsv = np.sqrt((A+B)/3)
Vmoins = (-Gam/2 - deltamoinsv)/k
Tmoins = Vmoins**2*m_Ca/(2*kb)
Vplus = (-Gam/2 + deltamoinsv)/k
Tplus = Vplus**2*m_Ca/(2*kb)

print(Tmoins,Tplus)

print('In the simulation, the low temperature before injection is within range 3-5 mK')

0.1767354101888944 0.0
In the simulation, the low temperature before injection is within range 3-5 mK


# Second method with temperature variation known

Consider that the temperature variation is known. In our case, with a GiantMolecule with mass $10^6$ Da, charge $Q=ze=e$, injection energy $50$ eV injection velocity $v\approx 100$ m/s. In the cases where detection is delivered, the giant molecule loses 50meV, transfers enough energy to raise the cloud temperature from 1mK to 100mK approximately. For a given $\delta T = 100$ mK, it is possible to compute the $\Delta P$, for different initial temperatures.

In [105]:
# plot P = MB x DopProfile

vmin = -10 # from 
vmax = 10  # to
nv = 500  # resolution
Tinit = 0.005   # temperature
Tfinal = 0.1   # temperature
deltaT = Tfinal-Tinit
delta=-1*Gam/2

nunuinit, dopinit = prob_fluo(vmin, vmax, nv, Tinit, delta, k, Rab, Gam)
nunufinal, dopfinal = prob_fluo(vmin, vmax, nv, Tfinal, delta, k, Rab, Gam)

fig = plt.figure('MB x DopProfile two temperatures',clear='True')
ax = fig.add_subplot(111)
ax.plot(nunuinit, dopinit,label=f'{Tinit} K')
ax.plot(nunufinal, dopfinal,label=f'{Tfinal} K')

ax.set_xlabel(r'$\nu = k v$ [Hz]')
ax.set_ylabel(r'Probability')

ax.grid()
ax.legend(loc=3,fontsize=15)

tight_layout()

In [106]:
# plot P = MB x DopProfile

# ajouter delta en fct de T
# Sinon tracer en fonction du detuning delta

vmin = -10 # from 
vmax = 10  # to
nv = 500  # resolution
Tinit = 0.005   # temperature
Tfinal = 0.1   # temperature
deltaT = Tfinal-Tinit

nunuinit, dopinit = prob_fluo(vmin, vmax, nv, Tinit, delta, k, Rab, Gam)
nunufinal, dopfinal = prob_fluo(vmin, vmax, nv, Tfinal, delta, k, Rab, Gam)

temps = linspace(.001, 1, 500)  # temperatures
deltas = [-1*Gam/2] # detunings  np.linspace(0, 10, 250)*
fluo_diff = [] # Here is stored the integral MBxDopProfile over detunings
for i, temp in enumerate(temps):
    fluo_diff.append([])    
    for j, delta in enumerate(deltas):
        nunuinit, dopinit = prob_fluo(vmin, vmax, nv, temp, delta, k, Rab, Gam)
        nunufinal, dopfinal = prob_fluo(vmin, vmax, nv, temp+deltaT, delta, k, Rab, Gam)
        fluo_diff[i].append(sum(dopfinal)*dv-sum(dopinit)*dv) # *dv
        
T_lim = -0.5*hbar*abs(deltas[0])*(1+(2*delta/Gam))**2/(4*delta/Gam)/kb


fig = plt.figure('fluo_diff',clear='True')
ax = fig.add_subplot(111)
ax.plot(temps, fluo_diff)
# ax.vlines(T_lim,-0.0015,0.0015)

ax.set_xlabel(r'$T_{init}$ [K]')
ax.set_ylabel(r'$\Delta F$')

ax.grid()
# ax.legend(loc=3,fontsize=15)

tight_layout()

En jouant sur le detuning placer le nuage dans un état où il est tjrs cristallin mais pas loin de transitionner vers liquide.

Le passage de la molécule va finir le travail et le chauffage RF amplifier.

En manip varier le detuning et mesurer la fluo. Néanmoins si les calculs au dessus sont intéressants, ils ne prennent pas en compte la faible durée de vie de la fluo. Certes la difference de fluorescence peut être élevée, mais de très courte durée..

In [18]:
# Compute limit temperature
deltabis = -0.5*Gam
T_limbis = -0.5*hbar*Gam**2*(1+(2*deltabis/Gam)**2) / (4*deltabis)/kb
print(T_lim*1e3,'mK')

0.5175985367512157 mK


In [109]:
# Cold limit temperature
# against detuning

temps = np.linspace(.00055, .1, 1000)  # temperatures
deltas = np.linspace(0, 4, 100)*Gam # detunings
fig = plt.figure('coucou',clear='True')
ax = fig.add_subplot(111)
ax.plot(deltas*1e-6,T_limbis(-deltas)*1e3,'+') # deltas*1e-6,
ax.grid()
tight_layout()

NameError: name 'T_limbis' is not defined